In [1]:
from transformers4rec import torch as tr
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import config
import os
# ignore warnings
from merlin.core.dispatch import get_lib
from merlin.schema.tags import Tags

import time
import merlin.models.tf as mm
from tensorflow.keras.callbacks import EarlyStopping

from merlin.schema.tags import Tags
from nvtabular.ops import *

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np


2023-09-11 12:42:25.978897: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-11 12:42:26.002557: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-09-11 12:42:27.435789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-11 12:42:27.436030: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-11 12:42:27.436082: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-09-11 12:42:27.681891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-11 12:42:27.682043: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-11 12:42:27.682095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [2]:
dir = os.path.join(config.data_final,'time',)
train = Dataset(os.path.join(config.data_final,'transformer4rec','train.parquet'),engine="parquet",  part_size="1GB")
# valid = Dataset(os.path.join(config.data_final,'transformer4rec', 'valid.parquet/*.parquet'),engine="parquet", part_size="1GB")

In [8]:
schema = train.schema.select_by_tag(Tags.SEQUENCE)


In [4]:
path= os.path.join(config.data_raw_dir, 'image_embeddings_norm.parquet')
image_embeddings = cudf.read_parquet(path)

max_sequence_length, d_model = 64, 64
# Define input module to process tabular input-features and to prepare masked inputs
input_module = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=30,
    # continuous_projection=512,
    # categorical_projection=256,
    aggregation="concat",
    d_output=d_model,
    masking="mlm",
)
# Define Next item prediction-task 
prediction_task = tr.NextItemPredictionTask(weight_tying=True,)

# # Define the config of the XLNet Transformer architecture
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=8, n_layer=5, 
)

# # Get the end-to-end model 
model = transformer_config.to_torch_model(input_module, prediction_task)

In [ ]:
path= os.path.join(config.data_raw_dir, 'image_embeddings_norm.parquet')
image_embeddings = cudf.read_parquet(path)

max_sequence_length, d_model = 64, 64

# Define input module to process tabular input-features and to prepare masked inputs
input_module = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=30,
    # continuous_projection=512,
    # categorical_projection=256,
    aggregation="concat",
    d_output=d_model,
    masking="mlm",
)

# Define Next item prediction-task 
prediction_task = tr.NextItemPredictionTask(weight_tying=True,)
concat_module = tr.ConcatenateFeatures()
projection_module = nn.Linear(d_model + image_embeddings.shape[1], d_model)
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=8, n_layer=5, 
)
model = transformer_config.to_torch_model(input_module, prediction_task, concat_module, projection_module)

In [5]:
training_args = tr.trainer.T4RecTrainingArguments(
            output_dir="./tmp",
            max_sequence_length=30,
            data_loader_engine='merlin',
            num_train_epochs=200, 
            dataloader_drop_last=False,
            per_device_train_batch_size = 1024,
            per_device_eval_batch_size = 1024,
            learning_rate=0.0005,
            fp16=True,
            report_to = [],
            logging_steps=200
        )

In [6]:
trainer = tr.Trainer(
    model=model,
    args=training_args,
    schema=schema,
    compute_metrics=True)

Using cuda_amp half precision backend


In [7]:
start_time_window_index = 81
final_time_window_index = 104
import glob
import os
import config

train_window = 3
for time_index in range(start_time_window_index, final_time_window_index - train_window + 1):

    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + train_window

    # train paths for time_index time_index+1 time_index+2
    # eval paths for time_index+3
    train_paths = []
    for i in range(time_index,time_index + train_window):
        train_paths.append(os.path.join(config.data_final,'time', f"{i}/train.parquet"))

    # train_paths = glob.glob(os.path.join(config.data_final,'time', f"{time_index_train}/train.parquet"))
    eval_paths = glob.glob(os.path.join(config.data_final,'time' , f"{time_index_eval}/valid.parquet"))

    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.train_dataset_or_path = train_paths
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    # Evaluate on the following day
    trainer.eval_dataset_or_path = eval_paths
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 


***** Running training *****
  Num examples = 40960
  Num Epochs = 200
  Instantaneous batch size per device = 1024
  Total train batch size (w. parallel, distributed & accumulation) = 1024
  Gradient Accumulation steps = 1
  Total optimization steps = 8000
  Number of trainable parameters = 19128640


********************
Launch training for day 81 are:
********************



../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [101,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0],

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
from transformers4rec.torch.utils.examples_utils import fit_and_evaluate

OT_results = fit_and_evaluate(trainer, start_time_index=81 , end_time_index=103 ,input_dir=dir)



***** Running training *****
  Num examples = 18432
  Num Epochs = 50
  Instantaneous batch size per device = 1024
  Total train batch size (w. parallel, distributed & accumulation) = 1024
  Gradient Accumulation steps = 1
  Total optimization steps = 900
  Number of trainable parameters = 4590016



***** Launch training for day 81: *****


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
OT_results

{'indexed_by_time_eval_/next-item/avg_precision@10': [0.03776850923895836,
  0.06101464107632637,
  0.06300443410873413,
  0.06414159387350082,
  0.0738043487071991,
  0.0712265744805336,
  0.07425947487354279],
 'indexed_by_time_eval_/next-item/avg_precision@20': [0.04034682363271713,
  0.06427469849586487,
  0.0656658336520195,
  0.06738380342721939,
  0.07703784108161926,
  0.07450366765260696,
  0.0775429755449295],
 'indexed_by_time_eval_/next-item/ndcg@10': [0.050285592675209045,
  0.07817145437002182,
  0.08036240935325623,
  0.08172142505645752,
  0.09382010251283646,
  0.09025176614522934,
  0.09415555000305176],
 'indexed_by_time_eval_/next-item/ndcg@20': [0.06035696342587471,
  0.09041143208742142,
  0.09061067551374435,
  0.09400550276041031,
  0.10596819967031479,
  0.1027255728840828,
  0.10652413964271545],
 'indexed_by_time_eval_/next-item/recall@10': [0.09088776260614395,
  0.13241347670555115,
  0.13577640056610107,
  0.13737118244171143,
  0.15708637237548828,
  0.15

In [ ]:
import numpy as np
# take the average of metric values over time
avg_results = {k: np.mean(v) for k,v in OT_results.items()}
for key in sorted(avg_results.keys()): 
    print(" %s = %s" % (key, str(avg_results[key]))) 

 indexed_by_time_eval_/next-item/avg_precision@10 = 0.045750506627170937
 indexed_by_time_eval_/next-item/avg_precision@20 = 0.04888351877098498
 indexed_by_time_eval_/next-item/ndcg@10 = 0.060058122095854385
 indexed_by_time_eval_/next-item/ndcg@20 = 0.07206008599504181
 indexed_by_time_eval_/next-item/recall@10 = 0.106090246497289
 indexed_by_time_eval_/next-item/recall@20 = 0.153581236043702


 indexed_by_time_eval_/next-item/avg_precision@10 = 0.045750506627170937
 indexed_by_time_eval_/next-item/avg_precision@20 = 0.04888351877098498
 indexed_by_time_eval_/next-item/ndcg@10 = 0.060058122095854385
 indexed_by_time_eval_/next-item/ndcg@20 = 0.07206008599504181
 indexed_by_time_eval_/next-item/recall@10 = 0.106090246497289
 indexed_by_time_eval_/next-item/recall@20 = 0.153581236043702

In [ ]:
a

In [ ]:
import os
import torch
import cudf
from merlin.io import Dataset
from nvtabular import Workflow
import numpy as np
from merlin.systems.dag import Ensemble
from merlin.systems.dag.ops.pytorch import PredictPyTorch
from merlin.systems.dag.ops.workflow import TransformWorkflow
from merlin.table import TensorTable, TorchColumn
from merlin.table.conversions import convert_col

In [ ]:
# recsys_trainer.load_model_trainer_states_from_checkpoint(os.path.join(config.notebook_dir,'tmp','checkpoint-30000'), model=model)

In [ ]:
df = cudf.read_parquet(os.path.join(config.data_final,'time','104','train.parquet'), columns=model.input_schema.column_names)
table = TensorTable.from_df(df.iloc[:100])
for column in table.columns:
    table[column] = convert_col(table[column], TorchColumn)
model_input_dict = table.to_dict()

In [ ]:
topk = 20
model.top_k = topk

In [ ]:
model.eval()
traced_model = torch.jit.trace(model, model_input_dict, strict=True)

In [ ]:
model_input_dict['article_id_list__values']

tensor([36727, 36914,   500,  1953,   247,  3711,  6438,  2858,  9473,    28,
         2588, 16851,  2111,  2616, 11658,  6219, 16564,  6219, 16564, 11117,
        11117, 11117,  4204, 16136,  3885,  1635,  1459,  1108,  4359,  9539,
            3,    17,   706,  3446,   861,  9047,  7450,  3883,  3883,   160,
          160, 16336,  4910, 14324,  3739,   546,   567,  4546,  2250, 15083,
        13467, 11492,  1537,  1239,  1239,   944,  2315, 12771,   893,   612,
           70,    70,  2487, 27922,  8766, 10352,  7715,  6364,  5302,  2568,
         7777,  7352,  3538,  4271, 16129,    66,  7298,   752,  3675,  2445,
        10514,  5558, 15770, 19199,   819,    36,   623, 18735, 30661,   587,
          783,   783,   314,   314, 24511, 24511, 19199,  1638, 14497,  9960,
          861,  1649,  7763,   355,  1814, 14598,  3883,  3531,  3829, 15639,
        12344, 31619,  1648,  1100,  2939,  2076,  2447, 21638,  1506, 17060,
         4564,  2221,  3141, 17022, 10833, 12649, 12649, 12278, 

In [ ]:

model_input_dict['article_id_list__offsets']

tensor([  0,  14,  23,  32,  41,  51,  60,  67,  75,  83,  97, 103, 111, 117,
        124, 130, 141, 148, 160, 169, 186, 204, 210, 222, 231, 238, 251, 258,
        265, 271, 278, 291, 298, 308, 315, 322, 330, 338, 344, 352, 358, 367,
        374, 384, 390, 397, 412, 424, 430, 440, 446, 455, 465, 473, 481, 487,
        494, 500, 510, 522, 528, 536, 544, 551, 558, 566, 574, 591, 597, 604,
        610, 616, 624, 630, 640, 656, 662, 668, 675, 683, 690, 696, 705, 714,
        725, 739, 746, 757, 766, 774, 783, 798, 805, 811, 821, 829, 836, 842,
        849, 859, 865], device='cuda:0', dtype=torch.int32)

In [ ]:
wf = Workflow.load(os.path.join(config.data_final,'transformer4rec','workflow'))

In [ ]:
torch_op = wf.input_schema.column_names >> TransformWorkflow(wf) >> PredictPyTorch(
    traced_model, model.input_schema, model.output_schema
)

In [ ]:
ensemble = Ensemble(torch_op, wf.input_schema)

In [ ]:
ens_config, node_configs = ensemble.export(os.path.join(config.notebook_dir,'tmp','final_model_ensamble'))

ValueError: The registered dtype mapping for numpy doesn't contain type unknown.